In [41]:
#5. 3х–1+2х+7

class Operation():
        
    def __init__(self, input_nodes = []):
        "Intialize an Operation"
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
  
    def compute(self):
                
        pass
    
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var
    
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var
    
class Placeholder():
        
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)
        
class Variable():
        
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
        
class Graph():
       
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        "Sets this Graph instance as the Global Default Graph"
        global _default_graph
        _default_graph = self
        
g = Graph()
g.set_as_default()
a = Variable(3)
b = Variable(-1)
c = Variable(2)
d = Variable(7)

x = Placeholder()

o = multiply(a,x)
p = add(o,b)
r = multiply(c,x)
s = add(p,r)
t = add(s,d)        
        
import numpy as np

def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

class Session:
    
    def run(self, operation, feed_dict = {}):
        "operation: The operation to compute"
        "feed_dict: Dictionary mapping placeholders to input values (the data)"
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                    
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output
    
sess = Session()
    
result = sess.run(operation=t,feed_dict={x:[1, 5, 10, 37, 100]})
    
result

array([ 11,  31,  56, 191, 506])